In [1]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [2]:
from cnn.cnn import load_dataset, create_model, steps
from dataset import dataset, load_files
from utilities.labels import LABELS
from dataset.data_augmentation import augmentated_filenames2

/home/congyu/congyu_program/pythons/forks/ECGNeuralNetwork


In [3]:
data_dir = "./data/mit_bih/"
save_dir = "./data/beats_img/"
_dataset_dir = './data/beats_img'
_model = './trained_models/cnn_aami_big_compare_cross.h5'

In [4]:
train = None
validation = None
augmentation = False
_batch_size = 32
_size = (64, 64)
_epochs = 50

In [5]:
train, validation, test = dataset.load_files_cross_patient(_dataset_dir, 
                                             verbose=True, 
                                             keep_ratio=1)
    
print(len(train))
print(len(validation))
print(len(test))

load DS tvt from ./data/beats_img
71637
17910
86109


In [6]:
model = create_model(out_dim=4)
# model.summary()

In [7]:


if augmentation:
    train = augmentated_filenames2(train)

callbacks_list = [ModelCheckpoint(_model, monitor='val_loss', save_best_only=False),
                  EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
                                baseline=None, restore_best_weights=False),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=0, mode='auto')
# TrainValTensorBoard(write_graph=False)
]

data_gen_train = load_dataset(train, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

data_gen_valid = load_dataset(validation, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

model.fit_generator(data_gen_train,
                    steps_per_epoch=steps(train, _batch_size),
                    epochs=_epochs,
                    validation_data=data_gen_valid,
                    validation_steps=steps(validation, _batch_size),
                    callbacks=callbacks_list)


Epoch 1/50
2239/2238 [==============================] - 223s 99ms/step - loss: 0.1867 - accuracy: 0.9683 - val_loss: 0.4530 - val_accuracy: 0.9455
Epoch 2/50
2239/2238 [==============================] - 214s 96ms/step - loss: 0.1095 - accuracy: 0.9830 - val_loss: 0.6704 - val_accuracy: 0.9461
Epoch 3/50
2239/2238 [==============================] - 190s 85ms/step - loss: 0.0954 - accuracy: 0.9862 - val_loss: 0.0899 - val_accuracy: 0.9461
Epoch 4/50
2239/2238 [==============================] - 198s 89ms/step - loss: 0.0876 - accuracy: 0.9875 - val_loss: 0.0571 - val_accuracy: 0.9787
Epoch 5/50
2239/2238 [==============================] - 195s 87ms/step - loss: 0.0839 - accuracy: 0.9885 - val_loss: 0.1431 - val_accuracy: 0.9461
Epoch 6/50
2239/2238 [==============================] - 298s 133ms/step - loss: 0.0826 - accuracy: 0.9885 - val_loss: 0.0664 - val_accuracy: 0.9816
Epoch 7/50
2239/2238 [==============================] - 277s 124ms/step - loss: 0.0787 - accuracy: 0.9897 - val_loss

In [12]:
from sklearn.metrics import f1_score

In [8]:
from keras.models import load_model

model = load_model(_model)

In [9]:
from evaluation import evaluate_cnn

In [17]:
y_true, y_pred = evaluate_cnn(model, label_type="AAMI", cross_patient=True, keep_ratio=1)
print("")

changing 86109 images to arrays
predicting on 86109 samples
precision:  0.9340131613569144
recall:  0.9285556678163722
f1score:  0.9300953193120629
macro f1 score 0.4611162607270816
micro f1 score 0.9285556678163722
              precision    recall  f1-score   support

           F       0.04      0.09      0.05       388
           N       0.97      0.96      0.96     80451
        SVEB       0.13      0.08      0.10      2049
         VEB       0.62      0.89      0.73      3221

    accuracy                           0.93     86109
   macro avg       0.44      0.51      0.46     86109
weighted avg       0.93      0.93      0.93     86109




In [18]:
print("macro f1 score", f1_score(y_true, y_pred, average='macro'))
print("micro f1 score", f1_score(y_true, y_pred, average='micro'))

macro f1 score 0.4611162607270816
micro f1 score 0.9285556678163722
